## Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)
pd.set_option("display.float_format", lambda x: '%.3f' % x)


# Import Dataset

In [3]:
online_retail = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/online_retail_II.xlsx",sheet_name="Year 2009-2010")
df = online_retail.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


## general information about dataset

In [4]:
def check_df(dataframe,head=5):
  print("################################ Head ################################\n")
  print(dataframe.head(head))
  print("################################ Tail ################################\n")
  print(dataframe.tail(head))
  print("################################ Shape ################################\n")
  print(dataframe.shape)
  print("################################ Types ################################\n")
  print(dataframe.dtypes)
  print("################################ NA ################################\n")
  print(dataframe.isnull().sum())
  print("################################ Qurtiles ################################\n")
  print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

################################ Head ################################

  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00  6.950    13085.000  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00  6.750    13085.000  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00  6.750    13085.000  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00  2.100    13085.000  United Kingdom
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24 2009-12-01 07:45:00  1.250    13085.000  United Kingdom
################################ Tail ################################

       Invoice StockCode                         Description  Quantity         InvoiceDate  Price  Customer ID        

In [5]:
df["Description"].nunique() # eşsiz ürün sayısı

4681

In [6]:
df["Description"].value_counts().head() # her üründen kaçar tane satılmış

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

In [7]:
df.groupby(["Description"]).agg({'Quantity':'sum'}).sort_values(by="Quantity",ascending=False).head() # en fazla hangi üründen kaç tane sayılmış

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [8]:
df["Invoice"].nunique() # eşsiz fatura sayısı

28816

### Calculate Total Price for any quantity

In [9]:
df["TotalPrice"] = df["Quantity"] * df["Price"] # her bir ürünün fiyat bilgisi

In [10]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [11]:
df.groupby(["Invoice"]).agg({'TotalPrice':'sum'}).head() # fatura başına toplam kazanılan miktar

,TotalPrice
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240


# Data Preprocessing

outlier detection yapılması RFM analizinde pekte mantıklı değil çünkü ne kaadar outlier leri baskılarsanız baskılayın zaten 5 puana denk gelceği için gerek yok böyle bir işleme, zaten aykırı değerler 5 puan alacaktır.

In [12]:
df.isnull().sum() # boş değerleri silmemiz lazım çünkü Customer İD gibi segmentasyon yapacağımız kısım boş olmaması lazım

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

In [13]:
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
TotalPrice     0
dtype: int64

In [15]:
df.describe().T # gördüğünüz üzere min, max sütunlarımızda eksili değerler var. bunlar iade edilen ürünlerden kaynaklıdır.
                # dolayısıyla onlar kaldırmamız gerekir

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.000,12.759,101.220,-9360.000,2.000,4.000,12.000,19152.000
Price,417534.000,3.888,71.132,0.000,1.250,1.950,3.750,25111.090
Customer ID,417534.000,15360.645,1680.811,12346.000,13983.000,15311.000,16799.000,18287.000
TotalPrice,417534.000,19.994,99.916,-25111.090,4.250,11.250,19.350,15818.400


In [16]:
df = df[~df["Invoice"].str.contains("C",na=False)]

In [17]:
df["InvoiceDate"].max() # en son satın alınan tarihi belirleriz.
                        # daha sonrasında üzerine 1-2 gün koyarak analiz tarihi sanki o günde yapılıyormuşcasına düşünürüz.

Timestamp('2010-12-09 20:01:00')

In [18]:
today_date = dt.datetime(2010,12,11)
today_date

datetime.datetime(2010, 12, 11, 0, 0)

### Calculating RFM Metrics

In [19]:
rfm = df.groupby(["Customer ID"]).agg({'InvoiceDate':lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                       'Invoice': lambda Invoice: Invoice.nunique(),
                                       'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [20]:
rfm.columns = ["recency","frequency","monetary"]

In [21]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [22]:
rfm = rfm[rfm["monetary"]>0] # monetary değeri sıfırdan fazla olanları rfm analizine dahil etmemiz daha mantıklı olacaktır.

In [23]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [24]:
rfm.shape # 4312 müşterinin Recency, Frequency ve Monetary değerlerini gösteriyor

(4312, 3)

### Calculating RFM Scores

In [25]:
rfm["recency_score"] = pd.qcut(rfm["recency"],5,labels=[5,4,3,2,1])

In [26]:
# rank burada ne işe yarıyor? örneğin birden fazla aralık aynı yere denk geldiği için ilk gördüğünü sınıfa atasın anlamına geliyor.
# örnek 5 parçaya bölmüşüz ilk parçaya 1 ler var ikinci parçayada denk gelcek şekilde birler var bu ilk gördüğü birleri birinci sınıfa diğerlerini ikinci sınıfa atasın anlamındadır.
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"),5,labels=[1,2,3,4,5])

In [27]:
rfm["monetary_score"] = pd.qcut(rfm["monetary"],5,labels=[1,2,3,4,5])

In [28]:
rfm["RFM_SCORE"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)

In [29]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.000,165,11,372.860,2,5,2,25
12347.000,3,2,1323.320,5,2,4,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,3,5,33
12351.000,11,1,300.930,5,1,2,51


In [30]:
rfm[rfm["RFM_SCORE"] == "55"] # şampiyonlar grubunu gözlemledik

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55
...,...,...,...,...,...,...,...
18225.000,1,15,7545.140,5,5,5,55
18226.000,14,15,6650.830,5,5,5,55
18229.000,2,10,3526.810,5,5,5,55


In [31]:
rfm[rfm['RFM_SCORE'] == "11"] # kaybedebileceğimiz grup

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12355.000,203,1,488.210,1,1,2,11
12362.000,374,1,130.000,1,1,1,11
12366.000,269,1,500.240,1,1,2,11
12368.000,264,1,917.700,1,1,3,11
12378.000,198,1,1407.700,1,1,4,11
...,...,...,...,...,...,...,...
15928.000,292,1,293.530,1,1,2,11
15929.000,280,1,594.000,1,1,3,11
15941.000,273,1,405.000,1,1,2,11


### Set RFM Names

In [32]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_to_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loaylists',
    r'5[4-5]': 'champions'
}

rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map,regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,5,2,25,cant_loose
12347.000,3,2,1323.320,5,2,4,52,potential_loaylists
12348.000,74,1,222.160,2,1,1,21,hibernating
12349.000,43,3,2671.140,3,3,5,33,need_to_attention
12351.000,11,1,300.930,5,1,2,51,new_customers


In [33]:
rfm[["segment","recency","frequency","monetary"]].groupby(["segment"]).agg(["mean","count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_to_attention    53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loaylists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [34]:
new_df = pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index
new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

In [35]:
new_df.to_csv("new_customers.csv")

In [36]:
rfm.to_csv("rfm.csv")

## writing the all process as a single function

In [ ]:
def create_rfm(dataframe,csv=False):
  # Data Preprocessing
  dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
  dataframe.dropna(inplace=True)
  dataframe = dataframe[~dataframe["Invoice"].str.contains("C",na=False)]
  dataframe = dataframe[(dataframe["Quantity"] > 0)]

  # Calculating RFM Metrics
  today_date = dt.datetime(2011,12,11)
  rfm = dataframe.groupby(["Customer ID"]).agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                                'Invoice': lambda Invoice: Invoice.nunique(),
                                                'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
  rfm.columns = ["recency","frequency","monetary"]
  rfm = rfm[rfm["monetary"]>0]

  # Calculating RFM Scores
  rfm["recency_score"] = pd.qcut(rfm["recency"],5,labels=[5,4,3,2,1])
  rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"),5,labels=[1,2,3,4,5])
  rfm["monetary_score"] = pd.qcut(rfm["monetary"],5,labels=[1,2,3,4,5])
  rfm["RFM_SCORE"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)

  # Set RFM Names
  seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_to_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loaylists',
    r'5[4-5]': 'champions'
  }
  rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map,regex=True)
  rfm.index = rfm.index.astype(int)

  # Save Results to CSV File
  if csv:
    rfm.to_csv("rfm.csv")

  return rfm

In [ ]:
df = online_retail.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [ ]:
create_rfm(df,csv=True)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346,530,11,372.860,2,5,2,25,cant_loose
12347,368,2,1323.320,5,2,4,52,potential_loaylists
12348,439,1,222.160,2,1,1,21,hibernating
12349,408,3,2671.140,3,3,5,33,need_to_attention
12351,376,1,300.930,5,1,2,51,new_customers
...,...,...,...,...,...,...,...,...
18283,383,6,641.770,4,5,3,45,loyal_customers
18284,432,1,461.680,3,2,2,32,about_to_sleep
18285,661,1,427.000,1,2,2,12,hibernating
